<a href="https://colab.research.google.com/github/robot-ai-working-group/google-colaboratory-env/blob/main/OpenAiGym_MountainCar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Open Ai Gym on Google colaboratry

## 環境作成

### library install


In [ ]:
!apt-get -qq -y install libcusparse8.0 libnvrtc8.0 libnvtoolsext1 > /dev/null
!ln -snf /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so.8.0 /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so
 
!apt-get -qq -y install xvfb freeglut3-dev ffmpeg> /dev/null
!pip install pyglet
!pip install pyopengl
!pip install pyvirtualdisplay
!pip install gym[classic_control]

E: Unable to locate package libcusparse8.0
E: Couldn't find any package by glob 'libcusparse8.0'
E: Couldn't find any package by regex 'libcusparse8.0'
E: Unable to locate package libnvrtc8.0
E: Couldn't find any package by glob 'libnvrtc8.0'
E: Couldn't find any package by regex 'libnvrtc8.0'


### virtual displayの設定


In [ ]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1024, 768))
display.start()
import os
 
 
# Error になるのでコメント化
# os.environ["DISPLAY"] = ":" + str(display.display) + "." + str(display.screen)

## Google Drive へマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### データ保存用ディレクトリの作成

In [ ]:
%mkdir "/content/drive/My Drive/Colab Notebooks/OpenAiGym"

mkdir: cannot create directory ‘/content/drive/My Drive/Colab Notebooks/OpenAiGym’: File exists


In [ ]:
%cd /content/drive/My Drive/Colab Notebooks/OpenAiGym

/content/drive/My Drive/Colab Notebooks/OpenAiGym


## git 連携



In [ ]:
!git clone https://github.com/robot-ai-working-group/google-colaboratory-env.git


Cloning into 'google-colaboratory-env'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.


In [ ]:
!git config --global user.email "yadamon4inoue@gmail.com"
!git config --global user.name "peeeechi"

## Sample を動かす

In [ ]:
import gym
import numpy as np
import json, os
 
 
def get_status(_observation):
    env_low = env.observation_space.low # 位置と速度の最小値
    env_high = env.observation_space.high # 位置と速度の最大値
    env_dx = (env_high - env_low) / 40 # 40等分
    # 0〜39の離散値に変換する
    position = int((_observation[0] - env_low[0])/env_dx[0])
    velocity = int((_observation[1] - env_low[1])/env_dx[1])
    return position, velocity
 
def update_q_table(_q_table, _action,  _observation, _next_observation, _reward, _episode):
 
    alpha = 0.2 # 学習率
    gamma = 0.99 # 時間割引き率
 
    # 行動後の状態で得られる最大行動価値 Q(s',a')
    next_position, next_velocity = get_status(_next_observation)
    next_max_q_value = max(_q_table[next_position][next_velocity])
 
    # 行動前の状態の行動価値 Q(s,a)
    position, velocity = get_status(_observation)
    q_value = _q_table[position][velocity][_action]
 
    # 行動価値関数の更新
    _q_table[position][velocity][_action] = q_value + alpha * (_reward + gamma * next_max_q_value - q_value)
 
    return _q_table
 
def get_action(_env, _q_table, _observation, _episode):
    epsilon = 0.002
    if np.random.uniform(0, 1) > epsilon:
        position, velocity = get_status(observation)
        _action = np.argmax(_q_table[position][velocity])
    else:
        _action = np.random.choice([0, 1, 2])
    return _action

In [ ]:
def writeParam(filename, param):
    with open(filename, 'w', encoding='utf-8') as wf:
        json.dump(param, wf, indent=2)

def readParam(filename):
    with open(filename, 'r', encoding='utf-8') as rf:
        param = json.load(rf)
    
    return param

In [ ]:
env = gym.make('MountainCar-v0')

paramFile = "param.json"


if (os.path.exists(paramFile)):
    array = np.array(readParam(paramFile))
    q_table =  array.reshape([40, 40, 3])
else:
    # Qテーブルの初期化
    q_table = np.zeros((40, 40, 3))
 
observation = env.reset()
rewards = []
 
# 10000エピソードで学習する
for episode in range(10000):
 
    total_reward = 0
    observation = env.reset()
 
    for _ in range(200):
 
        # ε-グリーディ法で行動を選択
        action = get_action(env, q_table, observation, episode)
 
        # if episode%1000 == 0:
            # env.render()
 
        # 車を動かし、観測結果・報酬・ゲーム終了FLG・詳細情報を取得
        next_observation, reward, done, _ = env.step(action)
 
        # Qテーブルの更新
        q_table = update_q_table(q_table, action, observation, next_observation, reward, episode)
        total_reward += reward
 
        observation = next_observation
 
        if done:
            # doneがTrueになったら１エピソード終了
            if episode%100 == 0:
                print('episode: {}, total_reward: {}'.format(episode, total_reward))
            rewards.append(total_reward)
            writeParam(filename=paramFile, param=list(q_table.flatten()))
            break

episode: 0, total_reward: -131.0
episode: 100, total_reward: -138.0
episode: 200, total_reward: -136.0
episode: 300, total_reward: -134.0
episode: 400, total_reward: -137.0
episode: 500, total_reward: -130.0
episode: 600, total_reward: -120.0
episode: 700, total_reward: -135.0
episode: 800, total_reward: -140.0
episode: 900, total_reward: -114.0
episode: 1000, total_reward: -137.0
episode: 1100, total_reward: -140.0
episode: 1200, total_reward: -132.0
episode: 1300, total_reward: -137.0
episode: 1400, total_reward: -143.0
episode: 1500, total_reward: -134.0
episode: 1600, total_reward: -131.0
episode: 1700, total_reward: -112.0
episode: 1800, total_reward: -138.0
episode: 1900, total_reward: -110.0
episode: 2000, total_reward: -135.0
episode: 2100, total_reward: -117.0
episode: 2200, total_reward: -136.0
episode: 2300, total_reward: -121.0
episode: 2400, total_reward: -200.0
episode: 2500, total_reward: -113.0
episode: 2600, total_reward: -132.0
episode: 2700, total_reward: -121.0
epis

# 新しいセクション

In [ ]:
!git add -A

hint: You've added another git repository inside your current repository.
hint: Clones of the outer repository will not contain the contents of
hint: the embedded repository and will not know how to obtain it.
hint: If you meant to add a submodule, use:
hint: 
hint: 	git submodule add <url> OpenAiGym-on-GoogleColabratory
hint: 
hint: If you added this path by mistake, you can remove it from the
hint: index with:
hint: 
hint: 	git rm --cached OpenAiGym-on-GoogleColabratory
hint: 
hint: See "git help submodule" for more information.


# 新しいセクション